# Developing multiple-shooting in Sun-B1 dynamics

2022.11.23

In [2]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
import joptimise
using Printf
using JSON

In [3]:
#include("../../julia-r3bp/R3BP/src/R3BP.jl")
using R3BP
include("../src/SailorMoon.jl")   # relative path to main file of module

┌ Info: Precompiling R3BP [a95c85f6-2be7-4e43-a066-d2c46f08437e]
└ @ Base loading.jl:1423
┌ Warning: Package R3BP does not have JSON in its dependencies:
│ - If you have R3BP checked out for development and have
│   added JSON as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with R3BP
└ Loading JSON into R3BP from project dependency, future warnings for R3BP are suppressed.
┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector}}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs/Docs.jl:240


Main.SailorMoon

In [23]:
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase and PlotlyKaleido need to be installed.
└ @ Plots /home/yuri/.julia/packages/Plots/530RA/src/backends.jl:319


Plots.PlotlyBackend()

In [12]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [13]:
moon = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth = plot_circle(param3b.mu2, param3b.as, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0)

2×100 Matrix{Float64}:
 389.981  389.978      389.971    389.96      …  389.978      389.981
   0.0      0.0735331    0.14677    0.219416      -0.0735331   -2.83969e-16

In [4]:
# solver settings within fitness function
# https://diffeq.sciml.ai/stable/solvers/dynamical_solve/#Symplectic-Integrators
#method = RK4()  # CalvoSanz4()
#reltol = 1e-10
#abstol = 1e-10
dt = 0.001

param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

Halo guess Az_km: 1200.0
Linear stability ν = 618.7618471048772


In [18]:
tmax_si = 280e-3 * 4   # N
isp_si = 1800   # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2500  # kg
rp_parking = (6378+200)/param3b.lstar   # parking orbit radius

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

0.009531869635160265

In [105]:
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
ωb = 2*pi*(t_synodic - t_sidereal)/t_synodic/t_sidereal*param3b.tstar
ωM = 2*pi / (t_synodic / param3b.tstar)
θM = π  # placeholder
params = [
    param3b.mu2, param3b.mus, param3b.as, θM, ωM, ωb, 0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_sun_dir_angles
]

#_prob_base = ODEProblem(R3BP.rhs_bcr4bp_thrust!, rand(7), [0,1], params);
u0_test = [
     387.6985504060269
  -2.7938137447835705e-7
  -5.620290959516347e-23
   1.0294967930424787e-6
  -1.203809978563231
   1.3395041586755617e-31
   1.0
]
_prob_base = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, u0_test, [0,-23.355], params);

In [106]:
sol = SailorMoon.integrate_rk4(_prob_base, 0.005);

In [107]:
pcart = plot(hcat(sol.u...)[1,:], hcat(sol.u...)[2,:], set_aspect=:equal, size=(700,700))
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

u_midpoint = sol.u[2500]
scatter!(pcart, [u_midpoint[1]], [u_midpoint[2]], marker=:circle)

pcart

In [108]:
@printf("Midpoint tf: %1.6f\n", sol.t[2500])
sol.u[2500]

Midpoint tf: -12.495000


7-element Vector{Float64}:
 390.02071608095923
  -3.9891203443226675
  -1.0506839914244052e-22
  -0.0930601491343078
   0.06533999100921835
  -2.1928917898975564e-23
   1.0

### Develop propagation function

In [109]:
n_arc = 5

5

In [110]:
# ballistic time right after launch
ballistic_time = 1*86400 / param3b.tstar
ballistic_time_back = 1*86400 / param3b.tstar

dt = 0.01
rp_parking = (6378+200)/param3b.lstar

0.017096890637263176

In [111]:
function unpack_x(x::AbstractVector{T}, verbose::Bool=false) where T
    # unpack
    nx = length(x)
    x_LEO = x[1:4+3n_arc]
    x_mid = x[5+3n_arc:13+9n_arc]    # x[5+3n_arc:4+3n_arc+9+6n_arc]
    x_LPO = x[14+9n_arc:17+12n_arc]  # x[14+9n_arc:13+9n_arc+4+3n_arc]
    
    # get time of flights
    tofs = [x_LEO[4], x_mid[8], x_mid[9], x_LPO[4]]
    θf = x_LPO[1]
    θs = [
        θf - param3b.oms*sum(broadcast(abs, tofs)),
        θf - param3b.oms*sum(broadcast(abs, tofs[3:4])),
        θf
    ]
    # print message
    if verbose
        @printf("ToF per arc  : %3.3f, %3.3f, %3.3f, %3.3f\n", tofs...)
        @printf("Phase angles : %3.3f, %3.3f, %3.3f\n", θs...)
    end
    return x_LEO, x_mid, x_LPO, tofs, θs
end

unpack_x (generic function with 2 methods)

In [317]:
function get_LEO_state(x_LEO, θs, verbose::Bool=false)
    # construct initial state
    sma = (rp_parking + x_LEO[1])/2
    ecc = (x_LEO[1] - rp_parking)/(x_LEO[1] + rp_parking)
    sv0_kep = [sma, ecc, 0.0, x_LEO[2], 0.0, 0.0]
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    sv0_em = SailorMoon.transform_earthIne_to_EMrot(
        sv0_i, θs[1], param3b.oml, param3b.mu2
    )
    #inertial2rotating(sv0_i, θs[1], 1.0) + [-param3b.mu2,0,0,0,0,0]
    # transform to Sun-B1 frame
    sv0_sunb1 = vcat(
        SailorMoon.transform_EMrot_to_SunB1(sv0_em, θs[1], param3b.oms, param3b.as),
        x_LEO[3]   # m0
    )
    if verbose
        println("SMA [km]: ", sma*param3b.lstar)
        println("ra  [km]: ", x_LEO[1]*param3b.lstar)
        println("Energy in inertial frame: ", -(param3b.mu2)/(2sma))
    end
    
    # ballistic propagation with small time-steps
    params = [
        param3b.mu2, param3b.mus, param3b.as, θs[1], ωM, ωb, 0.0, 0.0, 0.0, mdot, tmax,
        SailorMoon.dv_sun_dir_angles
    ]
    _prob = remake(_prob_base; tspan=[0,ballistic_time], u0 = sv0_sunb1, p = params)
    sol_ballistic_fwd = SailorMoon.integrate_rk4(_prob, 0.001);
    θ_iter = θs[1] + param3b.oms*sol_ballistic_fwd.t[end]
    return sol_ballistic_fwd.u[end], θ_iter, sol_ballistic_fwd
end

get_LEO_state (generic function with 2 methods)

In [318]:
param3b.mus

328900.5598102475

In [319]:
function get_LPO_state(x_LPO, θs, verbose::Bool=false)
    # propagate the periodic orbit until ϕ*Period.
    ϕ = x_LPO[2]
     x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))[:]
    _prob = remake(
        LPOArrival.prob_cr3bp_stm;
        tspan = (0.0, ϕ * LPOArrival.period),
        u0 = x0_stm,
        p=[param3b.mu2]
    )
    sol = SailorMoon.integrate_rk4(
        _prob, LPOArrival.dt
    )
    x_tf = sol.u[end][1:6]
    stm = transpose(reshape(sol.u[end][7:end], (6, 6)))

    # translate stable eigenvector and perturb final state
    ys = stm * LPOArrival.ys0
    if ys[1] > 0  # always perturb outward
        state_f = x_tf + LPOArrival.ϵ * ys/norm(ys)
    else
        state_f = x_tf - LPOArrival.ϵ * ys/norm(ys)
    end
    
    # transform to Sun-B1 frame
    svf_sunb1 = vcat(
        SailorMoon.transform_EMrot_to_SunB1(state_f, θs[3], param3b.oms, param3b.as),
        x_LPO[3]   # mf
    )
    # ballistic propagation with small time-steps
    params = [
        param3b.mu2, param3b.mus, param3b.as, θs[3], ωM, ωb, 0.0, 0.0, 0.0, mdot, tmax,
        SailorMoon.dv_sun_dir_angles
    ]
    _prob = remake(
        _prob_base; tspan=[0,-ballistic_time_back], 
        u0 = svf_sunb1, p = params
    )
    sol_ballistic_bck = SailorMoon.integrate_rk4(_prob, 0.001);
    θ_iter = θs[3] + param3b.oms*sol_ballistic_bck.t[end]
    
    if verbose
        println("svf_sunb1: \n", svf_sunb1)
    end
    return sol_ballistic_bck.u[end], θ_iter, sol_ballistic_bck
end

get_LPO_state (generic function with 2 methods)

In [320]:
function propagate_arc!(sv0, θ0, tspan, x_control, get_sols::Bool, sol_param_list, name::String)
    sv_iter = [el for el in sv0]
    θ_iter = 1*θ0
    for i = 1:n_arc
        τ, γ, β = x_control[1+3*(i-1) : 3*i]
        params = [
            param3b.mu2, param3b.mus, param3b.as, θM, ωM, ωb, τ, γ, β, mdot, tmax,
            SailorMoon.dv_sun_dir_angles
        ]
        _prob = remake(_prob_base; tspan=tspan, u0 = sv_iter, p = params)
        sol = SailorMoon.integrate_rk4(_prob, dt);
        #sol = DifferentialEquations.solve(_prob, RK4(), reltol=1e-10, abstol=1e-10)
        if get_sols
            push!(sol_param_list, [sol, params, name])
        end
        # update θ0 and sv0
        θ_iter += param3b.oms*sol.t[end]
        sv_iter = sol.u[end]
    end
    return sv_iter
end

propagate_arc! (generic function with 1 method)

In [321]:
multishoot_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false, verbose::Bool=false) where T
    # unpack decision vector
    x_LEO, x_mid, x_LPO, tofs, θs = unpack_x(x)
    
    # initialize storage
    sol_param_list = []
    
    # propagate from LEO forward
    sv0_LEO, θ0_leo, sol_ballistic_fwd = get_LEO_state(x_LEO, θs, verbose)
    svf_LEO = propagate_arc!(
        sv0_LEO, θ0_leo, [0, tofs[1]/n_arc], x_LEO[5 : end],
        get_sols, sol_param_list, "leo_arc"
    )
    
    # propagate midpoint backward
    svm0 = x_mid[1:7]  # state-vector at midpoint, [r,v,mass]
    svf_mid_bck = propagate_arc!(
        svm0, θs[2], [0, -tofs[2]/n_arc], x_mid[10 : end],
        get_sols, sol_param_list, "mid_bck_arc"
    )
    
    # propaagte midpoint forward
    svf_mid_fwd = propagate_arc!(
        svm0, θs[2], [0, tofs[3]/n_arc], x_mid[10+3n_arc : end],
        get_sols, sol_param_list, "mid_fwd_arc"
    )
    
    # propagate from LPO backward
    sv0_LPO, θ0_lpo, sol_ballistic_bck = get_LPO_state(x_LPO, θs, verbose)
    svf_lpo = propagate_arc!(
        sol_ballistic_bck.u[end], θ0_lpo, [0, -tofs[4]/n_arc], x_LPO[5 : end],
        get_sols, sol_param_list, "lpo_arc"
    )
    
    # residuals
    res = vcat(svf_mid_bck - svf_LEO, svf_lpo - svf_mid_fwd)[:]

    # output
    if get_sols == false
        return res
    else
        return res, sol_param_list, [sol_ballistic_fwd,sol_ballistic_bck], tofs
    end
end

#329 (generic function with 3 methods)

In [322]:
#sol.t[2500], sol.t[end]

In [329]:
sv_mid = [
 390.02071608095923
  -3.9891203443226675
  -1.0506839914244052e-22
  -0.0930601491343078
   0.06533999100921835
  -2.1928917898975564e-23
   1.0
]  # mid point state-vector

tof_leo2mid = 10.86
tof_mid2lpo = 12.495

# create test decision vector
τ_ig = 0.0
# x_LEO = [ra, Ω, m0, tof, controls...]
ig_x_LEO = vcat(
    [3.8, 5.9, 1.0, tof_leo2mid/2],
    vcat([[τ_ig,0,0] for i = 1:n_arc]...)
);
ig_x_mid = vcat(
    sv_mid, tof_leo2mid/2, tof_mid2lpo/2, 
    vcat([[τ_ig,0,0] for i = 1:2n_arc]...)
);
# x_LPO = [θf, ϕ, mf, tof, controls...]
ig_x_LPO = vcat(
    [0.24, 0.6, 1.0, tof_mid2lpo/2],
    vcat([[τ_ig,0,0] for i = 1:n_arc]...)
);
ig_x = vcat(ig_x_LEO, ig_x_mid, ig_x_LPO);
x_LEO, x_mid, x_LPO, tofs, θs = unpack_x(ig_x, true);

ToF per arc  : 5.430, 5.430, 6.247, 6.247
Phase angles : 21.848, 11.800, 0.240


In [330]:
@time hmp, sol_param_list, sols_ballistic, _ = multishoot_trajectory(ig_x, true, true);
hmp

SMA [km]: 734310.8135664859
ra  [km]: 1.4620436271329718e6
Energy in inertial frame: -0.003183200730227077
svf_sunb1: 
[387.694064484271, 0.3243975857666841, -4.424711876401422e-21, -0.22972474450874966, -0.935782260087894, 3.73543861430401e-21, 1.0]
  0.052069 seconds (1.36 M allocations: 28.036 MiB)


14-element Vector{Float64}:
 -8.380260368241352
  4.529051860131482
  3.126621945817898e-23
 -1.3419636415945368
  1.7081717221726467
 -2.585210153232183e-23
  0.0
  0.44950591349874003
  1.945174209005891
  7.882196291422873e-21
 -0.016378388652014603
  0.030121096433302186
 -6.757903040543394e-22
  0.0

In [331]:
arcs_color = Dict(
    "leo_arc" => :navy, 
    "mid_bck_arc" => :red1, 
    "mid_fwd_arc" => :darkorange, 
    "lpo_arc" => :green
)

pcart = plot(
    size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false
)
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

# ballistic legs
for sol_ballistic in sols_ballistic
    plot!(pcart, hcat(sol_ballistic.u...)[1,:], hcat(sol_ballistic.u...)[2,:], c=:dodgerblue, label=false)
end

# trajectory
for i = 1:length(sol_param_list)
    sol, _, name = sol_param_list[i]
    #if name == "mid_bck_arc" || name == "mid_fwd_arc" || name == "lpo_arc"
    plot!(pcart, SailorMoon.Array(sol)[1,:], SailorMoon.Array(sol)[2,:], 
        linewidth=1.5, label="$name", c=arcs_color[name])
    #end
end
# control node
scatter!(pcart, [x_mid[1]], [x_mid[2]], marker=:circle, color=:black, label="MP")
pcart